Итак, перед тем, как браться за статистику, нужно:

**1. Прочесть исходный файл и превратить его в структуру данных**

К заданию прилагается файл в формате csv, где все значения разделены запятыми. Это наши исходные данные. Чтобы применить к ним все возможности языка Python и библиотеки Pandas, надо импортировать эту библиотеку и сохранить её в переменной. По сокращённому названию панельных данных (panel data), с которых начиналась Pandas, эту переменную принято называть pd:

**import pandas as pd**

Для чтения csv-файла в библиотеке Pandas есть готовая функция — метод **read_csv()**. Как и все методы, он вызывается записью через точку после имени своего объекта. В скобках указывается аргумент (параметр) метода. У read_csv() это имя файла с данными. Прочтение превращает файл в структуру данных DataFrame. Имя переменной, в которой эта структура данных сохраняется, чаще всего df либо отражает тематику данных:

**df = pd.read_csv('music_log.csv')**

**2. Посмотреть на данные**

Вывести на экран таблицу и оценить данные:

print(df)
Как правило, таблица очень велика. Практичнее запросить определённое количество первых строк, методом head().

**3. Оценить качество предподготовки**

Нужно убедиться в том, что данные прошли предподготовку. По крайней мере, не должно быть пропусков и повторов. Пропущенные и неопределённые значения выявляет метод **isna()**, а суммарное количество таких значений — метод **sum()**.

**print(df.isna().sum())**


Повторяющиеся строки — дубликаты — выявляются методом **duplicated()** и подсчитываются тем же sum():

**print(df.duplicated().sum())**
Если возвращаются нули, всё хорошо — данные пригодны для исследования.


## Задача

1. Загрузите таблицу **exoplanet_catalog**, содержащую данные о планетах за пределами Солнечной системы, или экзопланетах. Сформируйте из данной таблицы новый датасет под назанием **exoplanet**, коорая будет содердать стоблцы name, mass, radius, discovered. Проверьте, чтобы название всех столбцов было корректным, при необходмости измените названия.

2. Удалите из получившейся таблица строки с нулевыми значениями.

3. Убедитесь, что таблица не содержит ни пропущенных значений, ни дубликатов.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Учеба/Lab3/exoplanet_catalog.csv')
stolb = ['name','mass','radius','discovered']
exoplanet = pd.DataFrame(data = df, columns= stolb)
exoplanet

exoplanet['name']=exoplanet['name'].fillna(0)
exoplanet['mass']=exoplanet['mass'].fillna(0)
exoplanet['radius']=exoplanet['radius'].fillna(0)
exoplanet['discovered']=exoplanet['discovered'].fillna(0)

exoplanet.drop_duplicates(inplace=True)
exoplanet.reset_index(inplace=True)
print(exoplanet)

print('Количество пропущенных значений\n',exoplanet.isnull().sum())
print('Количество дубликатов:',exoplanet.duplicated().sum())

      index  name   mass  radius  discovered
0         0   0.0   0.00    0.00      2008.0
1         1   0.0  21.00    0.00      2007.0
2         2   0.0   0.00    0.00      2009.0
3         4   0.0   0.00    0.00      2002.0
4         5   0.0   5.80    0.00      2006.0
...     ...   ...    ...     ...         ...
2500   4785   0.0   5.84    1.06      1996.0
2501   4791   0.0   0.62    0.00      1996.0
2502   4792   0.0   9.10    0.00      1999.0
2503   4793   0.0  23.58    0.00      1999.0
2504   4795   0.0  40.00    0.00      2014.0

[2505 rows x 5 columns]
Количество пропущенных значений
 index         0
name          0
mass          0
radius        0
discovered    0
dtype: int64
Количество дубликатов: 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Группировка данных

Слово «анализ» означает разбор, рассмотрение с разных сторон. Анализ данных начинают с разделения их на группы по какому-нибудь признаку. Эта операция называется группировка данных. Она помогает изучить материал более подробно, чтобы затем перейти к поиску взаимосвязей между отдельными группами.


Группировка оправданна, если данные чётко делятся по значимому признаку, а полученные группы близки к теме задачи. Например, когда есть данные обо всех покупках в супермаркете, можно смело заниматься группировкой. Так можно установить время наплыва покупателей и решить проблему пиковых нагрузок. Или посчитать средний чек — обычно для магазинов это ключевая метрика.


Стадии группировки хорошо укладываются в словесную формулу **split-apply-combine**:

•	разделить, **split** — разбиение на группы по определённому критерию;

•	применить, **apply** — применение какого-либо метода к каждой группе в отдельности, например, подсчёт численности группы методом count() или суммирование вызовом sum();

•	объединить, **combine** — сведение результатов в новую структуру данных, в зависимости от условий разделения и выполнения метода это бывает DataFrame и Series.

В библиотеке Pandas есть отличные инструменты группировки. Рассмотрим обращение с ними на примере анализа данных о планетах за пределами Солнечной системы, или экзопланетах. Орбитальные обсерватории засекли уже тысячи таких небесных тел. Их выявляют на снимках космических телескопов наши коллеги, аналитики данных. Поищем среди экзопланет похожие на Землю. Возможно, это наши будущие колонии, или там уже обитают разумные существа, с которыми однажды предстоит установить контакт.

DataFrame с данными по нескольким тысячам экзопланет сохранён в переменной **exoplanet**. Посмотрим на первые 30 строк таблицы:


In [ ]:
exoplanet.head(30)

,index,name,mass,radius,discovered
0,0,0.0,0.00,0.000000,2008.0
1,1,0.0,21.00,0.000000,2007.0
2,2,0.0,0.00,0.000000,2009.0
3,4,0.0,0.00,0.000000,2002.0
4,5,0.0,5.80,0.000000,2006.0
5,6,0.0,0.00,0.000000,1996.0
6,7,0.0,0.00,0.000000,2020.0
7,9,0.0,0.00,0.000002,2017.0
8,10,0.0,14.00,1.700000,2008.0
9,11,0.0,32.00,0.000000,2012.0


**Документация**

Столбцы:

•	name: название экзопланеты;

•	mass: масса в массах планеты Юпитер;

•	radius: радиус, пересчитанный в радиусах Земли;

•	discovered: год открытия экзопланеты.

*Источник: каталог экзопланет на портале exoplanet.eu*

На картинке изображен принцип **split-apply-combine** для таблицы с экзопланетами. Посмотрим, как вообще идут дела с поиском экзопланет. Сначала данные делят по группам, где каждая группа — это год. Потом метод **count()** подсчитывает численность каждой группы. В итоге получаем новую структуру данных с группами, где каждая содержит год и число открытых за этот год экзопланет.

![image.png](attachment:image.png)

В Рandas для группировки данных есть метод **groupby()**. Он принимает как аргумент название столбца, по которому нужно группировать. В случае с делением экзопланет по годам открытия:


**print(exoplanet.groupby('discovered'))**

**<pandas.core.groupby.DataFrameGroupBy object at 0x7fc1e1ca3400>**


Применение метода **groupby()** к объекту типа DataFrame приводит к созданию объекта особого типа — **DataFrameGroupBy**. Это сгруппированные данные. Если применить к ним какой-нибудь метод Pandas, они станут новой структурой данных типа **DataFrame** или **Series**.
Подсчитаем сгруппированные по годам экзопланеты методом **count()**:


In [ ]:
print(exoplanet.groupby('discovered').count())

            index  name  mass  radius
discovered                           
0.0             6     6     6       6
1988.0          1     1     1       1
1992.0          3     3     3       3
1995.0          3     3     3       3
1996.0          5     5     5       5
1998.0          2     2     2       2
1999.0          8     8     8       8
2000.0         15    15    15      15
2001.0          7     7     7       7
2002.0         12    12    12      12
2003.0          7     7     7       7
2004.0         16    16    16      16
2005.0         20    20    20      20
2006.0         23    23    23      23
2007.0         28    28    28      28
2008.0         38    38    38      38
2009.0         31    31    31      31
2010.0         76    76    76      76
2011.0        123   123   123     123
2012.0        107   107   107     107
2013.0        142   142   142     142
2014.0        382   382   382     382
2015.0        124   124   124     124
2016.0        523   523   523     523
2017.0      

Если нужно сравнить наблюдения по одному показателю, метод применяют к **DataFrameGroupBy** с указанием на один столбец. Нас в первую очередь интересует радиус экзопланет: мы ищем другую Землю. Давайте получим таблицу с единственным столбцом 'radius':

**exo_number = exoplanet.groupby('discovered')['radius'].count()**

**print(exo_number)**

In [ ]:
exo_number = exoplanet.groupby('discovered')['radius'].count()

print(exo_number)

discovered
0.0         6
1988.0      1
1992.0      3
1995.0      3
1996.0      5
1998.0      2
1999.0      8
2000.0     15
2001.0      7
2002.0     12
2003.0      7
2004.0     16
2005.0     20
2006.0     23
2007.0     28
2008.0     38
2009.0     31
2010.0     76
2011.0    123
2012.0    107
2013.0    142
2014.0    382
2015.0    124
2016.0    523
2017.0    119
2018.0    271
2019.0    144
2020.0    194
2021.0     75
Name: radius, dtype: int64


Получили Series, где по годам открытия расписано количество экзопланет, для которых удалось установить радиус.

Посмотрим, как меняется средний радиус открытых экзопланет год от года. Для этого надо сложить радиусы планет, открытых за определённый год, и поделить на их количество (которое мы уже нашли).

Сумма радиусов считается методом **sum()**:

**exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()**


**print(exo_radius_sum)**



In [ ]:
exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()

print(exo_radius_sum)

discovered
0.0         3.728100
1988.0      0.000000
1992.0      0.000000
1995.0      1.900000
1996.0      1.060000
1998.0      0.000000
1999.0      2.410000
2000.0      5.107000
2001.0      0.921000
2002.0      4.210000
2003.0      0.000000
2004.0      8.849700
2005.0      9.999000
2006.0     22.425000
2007.0     29.088600
2008.0     34.329000
2009.0     16.416794
2010.0     62.760660
2011.0     86.187374
2012.0     56.019507
2013.0     80.149100
2014.0    117.438230
2015.0     65.342690
2016.0    233.137600
2017.0     59.234622
2018.0    123.680070
2019.0     70.603372
2020.0     84.219100
2021.0     43.260410
Name: radius, dtype: float64


Очень кстати, что объекты Series можно делить друг на друга. Это позволит нам разделить перечень сумм радиусов на перечень количеств экзопланет без перебора в цикле:


**exo_radius_mean = exo_radius_sum/exo_number**

**print(exo_radius_mean)**


In [ ]:
exo_radius_mean = exo_radius_sum/exo_number

print(exo_radius_mean)

discovered
0.0       0.621350
1988.0    0.000000
1992.0    0.000000
1995.0    0.633333
1996.0    0.212000
1998.0    0.000000
1999.0    0.301250
2000.0    0.340467
2001.0    0.131571
2002.0    0.350833
2003.0    0.000000
2004.0    0.553106
2005.0    0.499950
2006.0    0.975000
2007.0    1.038879
2008.0    0.903395
2009.0    0.529574
2010.0    0.825798
2011.0    0.700710
2012.0    0.523547
2013.0    0.564430
2014.0    0.307430
2015.0    0.526957
2016.0    0.445770
2017.0    0.497770
2018.0    0.456384
2019.0    0.490301
2020.0    0.434119
2021.0    0.576805
Name: radius, dtype: float64


Точность наших приборов растёт, и новые экзопланеты по размерам всё ближе к Земле.

А теперь вернемся к анализу датафрейма с музыкльными предпочтениями слушателей Яндекс.Музыки. Идею объединения сервисов Музыка и Радио тестировали на небольшой группе пользователей. Результаты сведены в csv-файл, который вам предстоит изучить. Итог анализа таких данных — это метрики: величины, значения которых отражают пользовательские впечатления. Одна из важнейших — **happiness**. Здесь это среднее время, которое пользователь слушает музыку в течение выбранного периода времени (в нашей задаче — за сутки). Чем дольше пользователь слушает музыку, тем он довольнее. Ваша задача: найти значение happiness и посмотреть, как оно менялось.

Тем же методом **groupby()**, которым мы ищем новую Землю, можно поискать и необыкновенного человека в данных Яндекс.Музыки. Тем более, что без этого не выполнить поставленной менеджером задачи.

Прежде, чем рассчитывать метрику happiness, нужно изучить пользователей, чьё «счастье» мы собираемся оценить. Какие они, эти люди, которые слушают действительно много музыки? Есть ли у них особые предпочтения, или они потребляют всё подряд?


## Задача

1. Меломаны у нас есть. Сейчас узнаем идентификатор **user_id** одного из них. Для этого сгруппируем данные по каждому пользователю, чтобы собрать жанры прослушанных им композиций.

Сгруппируйте DataFrame по столбцу user_id, сохраните полученный результат в переменной genre_grouping.

Посчитайте количество жанров, которые выбрали пользователи, методом count(), указав, что выбираем один столбец genre_name.

Сохраните результат в переменной genre_counting и выведите первые 30 строк этой таблицы.

2. Быть может, те, кто за день слушает больше 50 песен, имеют более широкие предпочтения. Чтобы найти такого, изготовим универсальный инструмент.

Напишите функцию user_genres, которая принимает некую группировку как свой аргумент group. Функция должна перебирать группы, входящие в эту группировку.

В каждой группе два элемента — имя группы с индексом 0 и список значений с индексом 1.

Обнаружив такую группу, в которой список (элемент с индексом 1) содержит более 50 значений, функция возвращает имя группы (значение элемента с индексом 0).

3. Вызовите функцию **user_genres**, как аргумент передайте ей **genre_grouping**. Результат – **user_id** неведомого нам любителя музыки – сохраните в переменной **search_id** и выведите значение на экран.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Учеба/Lab3/yandex_music.csv')
new_names = ['user_id', 'track_name', 'artist_name', 'genre_name','total_play_seconds','time','day']

if len(new_names) == len(df.columns):
    df = df.set_axis(new_names, axis='columns')
else:
    print("Ошибка: количество новых имен не совпадает с количеством столбцов.")

genre_grouping = df.groupby('user_id')
genre_counting = df.groupby('user_id')['genre_name'].count()
genre_counting.head(30)

def user_genres(group):
  for i in group:
    if len(i[1]) > 50:
      return i[0]
  return None;

search_id = user_genres(list(genre_grouping))
print(search_id)

#Сколько слушают из Москвы и сколько слушают из Санкт-Петербурга хип-хоп.
genre_counting_M = df[(df['genre_name'] == 'hip') & (df['total_play_seconds']== 'Moscow')].count()
genre_counting_S = df[(df['genre_name'] == 'hip') & (df['total_play_seconds'] == 'Saint-Petersburg')].count()
print("Количество слушащих hip-hop в Москве:", genre_counting_M['user_id'])
print("Количество слушащих hip-hop в Санкт-Петербург:", genre_counting_S['user_id'])

414F229D
Количество слушащих hip-hop в Москве: 2156
Количество слушащих hip-hop в Санкт-Петербург: 992


# Сортировка данных

Поиск необычного в группе — что среди планет, что среди меломанов — это прежде всего поиск чемпионов: объектов с выдающимися показателями по разным статьям. Как всю таблицу, так и отдельные группы изучают, сортируя строки по какому-либо столбцу.

В Pandas для этой операции есть метод **sort_values()**. У него два аргумента:

• **by = 'имя столбца'** — имя столбца, по которому нужно сортировать;

• **ascending:** по умолчанию True. Для сортировки по убыванию установите значение False.

![image.png](attachment:image.png)


Среди экзопланет интересны близкие по размерам к Земле. Есть ли такие? Отсортируем список по радиусу в порядке возрастания. Тогда в голове таблицы окажутся самые малые, на которых гравитация не прижмёт нас к полу.

**print(exoplanet.sort_values(by='radius').head(30))**


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Учеба/Lab3/exoplanet_catalog.csv')
stolb = ['name','mass','radius','discovered']
exoplanet = pd.DataFrame(data = df, columns= stolb)
print(exoplanet.sort_values(by='radius').head(30))

      name     mass    radius  discovered
9      NaN      NaN  0.000002      2017.0
16     NaN      NaN  0.000062      2019.0
4357   NaN      NaN  0.000900      2019.0
3258   NaN      NaN  0.025430      2014.0
3207   NaN  0.01000  0.026000      2013.0
3354   NaN      NaN  0.035950      2015.0
1744   NaN  0.01353  0.042000      2014.0
3355   NaN      NaN  0.044000      2015.0
2126   NaN      NaN  0.046000      2016.0
2221   NaN  0.00021  0.047000      2014.0
3356   NaN      NaN  0.047000      2015.0
3611   NaN  0.01300  0.048000      2013.0
3357   NaN      NaN  0.049000      2015.0
3323   NaN  0.00300  0.051000      2011.0
1745   NaN  0.00944  0.052000      2014.0
2491   NaN      NaN  0.054000      2016.0
1839   NaN      NaN  0.054000      2016.0
1611   NaN      NaN  0.055000      2016.0
2212   NaN      NaN  0.057000      2016.0
2185   NaN      NaN  0.058000      2016.0
2439   NaN      NaN  0.058000      2016.0
2937   NaN      NaN  0.059000      2016.0
3337   NaN      NaN  0.060000     

Оказывается, некоторые из уже открытых экзопланет по размерам близки не то что к Земле, но уже и к Луне! Получим список экзопланет с радиусом меньше земного. Смотрите, как логический оператор (здесь это <) задействуется в отборе элементов столбца. Дальше нам этот приём не раз понадобится.


**print(exoplanet[exoplanet['radius'] < 1])**

In [ ]:
print(exoplanet[exoplanet['radius'] < 1])

      name      mass    radius  discovered
9      NaN       NaN  0.000002      2017.0
16     NaN       NaN  0.000062      2019.0
22     NaN  20.00000  0.920000      2010.0
41     NaN  43.00000  0.910000      2021.0
42     NaN  67.00000  0.970000      2021.0
...    ...       ...       ...         ...
4731   NaN       NaN  0.181110      2018.0
4732   NaN  26.00000  0.920000      2009.0
4734   NaN   0.62000  0.973000      2007.0
4751   NaN  59.30000  0.783000      2021.0
4783   NaN   0.01517  0.167190      2018.0

[2974 rows x 4 columns]


Но и этот список такой длинный, что изучать его лучше по частям. Экзопланеты, близкие по размерам к Земле, за последнее десятилетие открывали нередко. Можно изучать список открытых за каждый год. Например, для 2014 года (вновь обратите внимание на работу логического оператора, теперь это ==):

**print(exoplanet[exoplanet['discovered'] == 2014])**


In [ ]:
print(exoplanet[exoplanet['discovered'] == 2014])

      name  mass  radius  discovered
106    NaN   NaN     NaN      2014.0
222    NaN   NaN     NaN      2014.0
223    NaN   NaN     NaN      2014.0
233    NaN   NaN     NaN      2014.0
234    NaN   NaN     NaN      2014.0
...    ...   ...     ...         ...
4763   NaN   NaN     NaN      2014.0
4764   NaN   NaN     NaN      2014.0
4776   NaN   NaN     NaN      2014.0
4784   NaN   NaN     NaN      2014.0
4795   NaN  40.0     NaN      2014.0

[872 rows x 4 columns]


А чтобы не тратить время на лишнее, поставим оба условия сразу. Для этого в Pandas есть логический оператор &, подобный оператору and языка Python. Напомним, его смысл на русском языке можно передать словами «и ещё»:

**# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году**

**exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]**

**print(exo_small_14)**



In [ ]:
exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]

print(exo_small_14)

      name    mass  radius  discovered
400    NaN  0.7600   0.944      2014.0
465    NaN  0.2370   0.912      2014.0
477    NaN  0.3190   0.998      2014.0
1164   NaN  0.0372   0.226      2014.0
1690   NaN  1.4500   0.940      2014.0
...    ...     ...     ...         ...
4039   NaN  0.0410   0.305      2014.0
4051   NaN  0.0270   0.238      2014.0
4065   NaN  0.0110   0.132      2014.0
4077   NaN  0.0110   0.178      2014.0
4088   NaN  0.0190   0.132      2014.0

[769 rows x 4 columns]


Отсортируем результат в порядке убывания радиуса.

**print(exo_small_14.sort_values(by = 'radius', ascending = False))**


In [ ]:
print(exo_small_14.sort_values(by = 'radius', ascending = False))

      name     mass   radius  discovered
477    NaN  0.31900  0.99800      2014.0
3329   NaN  0.25000  0.97800      2014.0
3319   NaN  2.50000  0.96000      2014.0
400    NaN  0.76000  0.94400      2014.0
1690   NaN  1.45000  0.94000      2014.0
...    ...      ...      ...         ...
2263   NaN      NaN  0.06200      2014.0
1745   NaN  0.00944  0.05200      2014.0
2221   NaN  0.00021  0.04700      2014.0
1744   NaN  0.01353  0.04200      2014.0
3258   NaN      NaN  0.02543      2014.0

[769 rows x 4 columns]


Самая крупная планета, Kepler 106 d – почти как Земля, вращается вокруг звезды Kepler 106 в созвездии Лебедя. Эта звезда очень похожа на наше Солнце. Правда, до неё 1435 световых лет — далековато. Но, возможно, учёные что-нибудь придумают. А мы пока применим эту технологию к нашему бизнесу, в «приземлённой» задаче.
## Задача

1. Космический телескоп Kepler открыл похожую на Землю планету у похожей на Солнце звезды. А вы в данных Яндекс.Музыки обнаружили меломана с уникальными данными. Он за день послушал больше 50 композиций.
Получите таблицу с прослушанными им треками.

Для этого запросите из структуры данных df строки, отвечающие сразу двум условиям:

1) значение в столбце 'user_id' должно быть равно значению переменной search_id;

2) время прослушивания, т.е. значение в столбце 'total_play_seconds', не должно равняться 0.

Сохраните результат в переменной music_user.

2. Теперь узнаем, сколько времени он слушал музыку каждого жанра.

Сгруппируйте данные таблицы music_user по столбцу 'genre_name' и получите сумму значений столбца 'total_play_seconds'. Сохраните результат в переменной sum_music_user и выведите её значение на экран.

3. Кажется, предпочтения нашего меломана начинают проявляться. Но, возможно, длительность композиций от жанра к жанру сильно различается. Важно знать, сколько треков каждого жанра он включил.

Сгруппируйте данные по столбцу genre_name и посчитайте, сколько значений в столбце genre_name. Сохраните результат в переменной count_music_user и выведите её значение на экран.

4. Чтобы предпочтения были видны сразу, нужно крупнейшие значения расположить наверху. Отсортируйте данные в группировке sum_music_user по убыванию. Внимание: когда применяете метод sort_values() к Series с единственным столбцом, аргумент by указывать не нужно, только порядок сортировки.

Сохраните результат в переменной final_sum и выведите её значение на экран.

5. Теперь то же самое надо сделать с числом прослушанных меломаном композиций. Отсортируйте данные группировки count_music_user по убыванию. Сохраните результат в переменной final_count, значение которой выведите на экран.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Учеба/Lab3/yandex_music.csv')
new_names = ['user_id', 'track_name', 'artist_name', 'genre_name','total_play_seconds','time','day']

if len(new_names) == len(df.columns):
    df = df.set_axis(new_names, axis='columns')
else:
    print("Ошибка: количество новых имен не совпадает с количеством столбцов.")

genre_grouping = df.groupby('user_id')
genre_counting = df.groupby('user_id')['genre_name'].count()

def user_genres(group):
  for i in group:
    if len(i[1]) > 50:
      return i[0]
  return None;

search_id = user_genres(list(genre_grouping))

music_user = df[(df['user_id'] == search_id) & (df['total_play_seconds'] != 0)]

sum_music_user = df.groupby('genre_name')['total_play_seconds'].count()
print(sum_music_user)

count_music_user = df.groupby('genre_name').count()
print(count_music_user)

final_sum = sum_music_user.sort_values(ascending = False)
print(final_sum)



genre_name
acid              1
acoustic          6
action            4
adult            27
africa           17
               ... 
western         106
world          2069
worldbeat         2
ïîï               1
электроника       1
Name: total_play_seconds, Length: 289, dtype: int64
             user_id  track_name  artist_name  total_play_seconds  time   day
genre_name                                                                   
acid               1           1            1                   1     1     1
acoustic           6           6            2                   6     6     6
action             4           4            3                   4     4     4
adult             27          27           27                  27    27    27
africa            17          17           15                  17    17    17
...              ...         ...          ...                 ...   ...   ...
western          106         106          103                 106   106   106
world          

In [ ]:
final_count = count_music_user.sort_values(by = 'total_play_seconds', ascending = False)
print(final_count)

                  user_id  track_name  artist_name  total_play_seconds  time  \
genre_name                                                                     
pop                  8850        8712         8222                8850  8850   
dance                6761        6724         5439                6761  6761   
rock                 6192        6117         5953                6192  6192   
electronic           5852        5848         5179                5852  5852   
hip                  3148        3118         2892                3148  3148   
...                   ...         ...          ...                 ...   ...   
lovers                  1           1            1                   1     1   
loungeelectronic        1           1            1                   1     1   
laiko                   1           1            1                   1     1   
kayokyoku               1           1            1                   1     1   
электроника             1           1   